In [7]:
import os
import time
import pyautogui
import random
from dotenv import load_dotenv
import pyperclip
import pyanty
from web3 import Web3
from openai import OpenAI
import requests

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Warning! To work with pyppeteer.
 You need install it with: pip install pyppeteer
Warning! To work with playwright.
 You need install it with: pip install playwright


In [9]:
def get_full_xpath_element(driver, element):
    full_xpath_element= driver.execute_script(
        """function absoluteXPath(element) {
            var comp, comps = [];
            var parent = null;
            var xpath = '';
            var getPos = function(element) {
                var position = 1, curNode;
                if (element.nodeType == Node.ATTRIBUTE_NODE) {
                    return null;
                }
                for (curNode = element.previousSibling; curNode; curNode = curNode.previousSibling) {
                    if (curNode.nodeName == element.nodeName) {
                        ++position;
                    }
                }
                return position;
            };
            if (element instanceof Document) {
                return '/';
            }
            for (; element && !(element instanceof Document); element = element.nodeType == Node.ATTRIBUTE_NODE ? element.ownerElement : element.parentNode) {
                comp = comps[comps.length] = {};
                comp.name = element.nodeName;
                comp.position = getPos(element);
            }
            for (var i = comps.length - 1; i >= 0; i--) {
                comp = comps[i];
                xpath += '/' + comp.name.toLowerCase() + (comp.position > 1 ? '[' + comp.position + ']' : '');
            }
            return xpath;
        }
        return absoluteXPath(arguments[0]);""",
        element
    )
    return full_xpath_element

In [41]:
load_dotenv()

DOLPHIN_API = os.getenv("DOLPHIN_API")
TOWNS_LINK = os.getenv("TOWNS_LINK")
OPENAI_API = os.getenv("OPENAI_API")

In [43]:
api = pyanty.DolphinAPI(api_key=DOLPHIN_API)

In [46]:
# not necessary
response = api.get_profiles()
profile_id = [profile["id"] for profile in response["data"]][15]

In [49]:
# open profile with selenium (FINISHED)

In [51]:
def open_profile_with_selenium(profile_id):
    response = pyanty.run_profile(profile_id)
    port = response['automation']['port']
    driver = pyanty.get_driver(port=port)
    
    return driver

driver = open_profile_with_selenium(profile_id)

In [19]:
# open profile with selenium END (FINISHED)

In [23]:
# open towns START (FINISHED)

In [53]:
def open_towns(driver):
    # open link
    driver.execute_script(f"window.open('https://app.towns.com/explore', '_blank');")
    driver.implicitly_wait(5)
    
    # make tab with Towns active 
    for handle in reversed(driver.window_handles):  # Iterate in reverse order
        driver.switch_to.window(handle)
        if "Towns" in driver.title:  # Adjust the title check if needed
            break  # Found the correct tab

    # wait till open
    element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//a[@href='/t/new'] | //*[contains(text(), 'Log In')]")))
    
    time.sleep(2)

    # return True if Login is needed
    return element.text == "Login"

login_need = open_towns(driver)
login_need

False

In [26]:
# open towns END (FINISHED)

In [217]:
# Twitter login START 

In [26]:
def login_twitter():
    # # find element with Login text
    # login_p_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Log In')]")))
    # # get xpath
    # login_p_xpath = get_full_xpath_element(driver, login_p_element)
    # # get xpath of button with this element
    # login_button_xpath = login_p_xpath[:login_p_xpath.rfind("button") + 6]
    # # get button element
    # login_button_element = driver.find_element(By.XPATH, login_button_xpath)
    # # click Login button
    # login_button_element.click()

    # find element with Login text
    login_button_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//button[contains(text(), 'Log In')]")))
    # click Login button
    login_button_element.click()
    
    # wait untill elements are loaded 
    WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'LoginMethodButton')]")))
    # find login_method elements
    login_elements = driver.find_elements(By.XPATH, "//button[contains(@class, 'LoginMethodButton')]")
    # get twitter method_element
    twitter_method_element = find_login_method_element(login_elements, "Twitter")
    # click
    twitter_method_element.click()
    
    # find authorize button
    autorize_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='OAuth_Consent_Button']")))
    # click
    autorize_element.click()
    
    # wait until successfull text will appear 
    success_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
        (By.XPATH, "//*[contains(text(), 'Successfully connected with Twitter')] | //*[contains(text(), 'Authentication failed')]")))

    if success_element.text == "Authentication failed":
        print(success_element.find_element(By.XPATH, "..").text)
        return False
    else:
        return True

def find_login_method_element(login_elements, method="Twitter"):
    # find twitter_login_method_element
    login_method_element = None
    for element in login_elements:
        if element.text == method:
            login_method_element = element
    
    return login_method_element

if login_need:
    login_twitter()

In [28]:
# Twitter login END (FINISHED)

In [33]:
# login google START

In [35]:
def login_google():
    # find element with Login text
    login_button_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[contains(text(), 'Log In')]")))
    # click Login button
    login_button_element.click()

    # wait till elements are visible
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'LoginMethodButton')]")))
    # find login_method elements
    login_elements = driver.find_elements(By.XPATH, "//button[contains(@class, 'LoginMethodButton')]")
    # get and click twitter method_element
    google_method_element = find_login_method_element(login_elements, "Google")
    google_method_element.click()

    # find and click first google account
    login_elements = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[@role='link' and @data-item-index='0']")))
    login_elements.click()

    # wait until successfull text will appear 
    success_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
        (By.XPATH, "//*[contains(text(), 'Successfully connected with Google')] | //*[contains(text(), 'Authentication failed')]")))

    if success_element.text == "Authentication failed":
        print(success_element.find_element(By.XPATH, "..").text)
        return False
    else:
        return True

if login_need:
    login_google()

In [37]:
# login google END (FINISHED)

In [ ]:
# reset caches START

In [272]:
def reset_caches():
    # find and click bug_report button
    bug_report_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//span[@data-testid='report-bug-button']")))
    bug_report_element.click()

    # find and click reset_caches button
    reset_caches_elements = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Reset Caches')]")))
    reset_caches_elements.click()
    
    # find and click confirm reset caches button
    confirm_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[contains(text(), 'Confirm')]")))
    confirm_element.click()

# reset_caches()

In [264]:
# reset caches END (FINISHED)

In [ ]:
# get connected wallet START

In [151]:
def get_connected_wallet():
    def copy_connected_wallet():
        # find element with Town Wallet <p>
        towns_wallet_text_element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//p[contains(text(), 'Towns Wallet')]")))
        # find and click on button that copy text
        copy_wallet_element = towns_wallet_text_element.find_element(By.XPATH, "..").find_element(By.XPATH, "span")
        copy_wallet_element.click()
        
        return pyperclip.paste()
        
    # find and click user-profile-button
    user_profile_button_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//span[@data-testid='user-profile-button']")))
    user_profile_button_element.click()

    # find and click Linked Wallets button
    linked_wallets_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Linked Wallets')]")))
    linked_wallets_element.click()

    # retrieve connected wallet until 2 consequtive wallets are identical
    copied_wallet = None
    prev_copied_wallet = None
    while True:
        copied_wallet = copy_connected_wallet()
        if copied_wallet == prev_copied_wallet:
            return copied_wallet
        else:
            prev_copied_wallet = copied_wallet

connected_wallet = get_connected_wallet()
connected_wallet

'0xBAFc33E259BDA12fdf5Bdf7e764508EBF772005e'

In [ ]:
# get connected wallet END

In [51]:
# open random town START (FINISHED)

In [153]:
def open_random_town(driver):
    # find create_new_town element 
    new_town_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//a[@href="/t/new"]')))
    new_town_xpath = get_full_xpath_element(driver, new_town_element)
    
    # find left_box element
    left_box_xpath = new_town_xpath[:new_town_xpath.rfind("div[") - 1]
    left_box_element = driver.find_element("xpath", left_box_xpath)
    
    # find all left_box elements
    left_box_elements = left_box_element.find_elements("xpath", "./div")
    towns_num = len(left_box_elements) - 2
    
    # choose random towns
    town_to_open = random.randint(0, towns_num - 1)
    
    # open random towns
    left_box_elements[town_to_open + 2].click()
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Share Town Link')]")))

open_random_town(driver)

In [155]:
# open random town END (FINISHED)

In [160]:
# join town START

In [170]:
def join_free_town(town_link):
    # open town link
    driver.get(town_link)

    # check if you already a member
    share_link_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
        (By.XPATH, "//*[contains(text(), 'Share Town Link')] | //*[contains(text(), 'Share Link')]")))
    if share_link_element.text == "Share Town Link":
        print("You are already a member of this town")
        return False

    # check if town is free
    membership_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Membership')]")))
    price_box_element = membership_element.find_element(By.XPATH, "..").find_element(By.XPATH, "..").find_element(By.XPATH, "..")
    if "Free" not in price_box_element.text:
        print("Town is not Free")
        return False

    # find and click JOIN button
    join_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[@type='button']")))
    join_element.click()

In [172]:
town_link = "https://app.towns.com/t/0xdc443716b9c203ff3ee9f3631182bfedf205dac8/"
join_free_town(town_link)

You are already a member of this town


False

In [86]:
# find and click JOIN button
join_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//button[contains(text(), 'Join')]")))
join_element.click()

In [100]:
# wait till nft minting and welcome page. Find and click Join Town
join_town_text_element = WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Join Town')]")))
join_town_text_xpath = get_full_xpath_element(driver, join_town_text_element)
join_town_button_xpath = join_town_text_xpath[:join_town_text_xpath.rfind("button") + 6]
join_town_button_element = driver.find_element(By.XPATH, join_town_button_xpath)
join_town_button_element.click()

In [ ]:
# join town END

In [ ]:
# join payd TOWN

In [110]:
def get_base_balance(address):
    # Replace with your Base RPC URL
    BASE_RPC_URL = "https://mainnet.base.org"

    # Connect to the Base network
    web3 = Web3(Web3.HTTPProvider(BASE_RPC_URL))
    
    # Check if the connection is successful
    if web3.is_connected():
        print("Connected to Base Network")
    else:
        print("Failed to connect")
    
    # Get the balance (returns balance in Wei)
    balance_wei = web3.eth.get_balance(address)
    # Convert Wei to Ether
    balance_eth = web3.from_wei(balance_wei, "ether")
    print(f"ETH Balance on Base: {balance_eth} ETH")
    
    return balance_eth

In [21]:
def get_random_town_link():
    # Read the file and get all links
    with open("towns_links.txt", "r", encoding="utf-8") as file:
        links = file.read().splitlines()  # Read lines and remove newline characters
    
    # Choose a random link
    if links:
        return random.choice(links)
    else:
        return None

def open_town():
    driver.get(get_random_town_link())
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Threads')]")))


In [ ]:
def check_available_channels():
    

In [ ]:
# open random channel START

In [180]:
def open_random_channel(driver):
    # find all possible channels
    channel_elements = driver.find_elements(By.XPATH, "//span[contains(@data-testid, '-channel')]")
    
    # choose some channel
    channel_to_open = random.randint(0, len(channel_elements)-1)
    
    # open this channel
    channel_elements[channel_to_open].click()
    
    # wait untill it is loaded
    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Send a message to ')]")))

open_random_channel(driver)

In [ ]:
# open random channel END

In [ ]:
# write a message START

In [165]:
# wait till page is loaded
new_message_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Send a message to ')]")))

In [131]:
# find messages
message_elements = driver.find_elements(By.XPATH, "//div[@data-testid='chat-message-container']")

In [163]:
def get_last_n_messages(message_elements):
    messages = list()
    errors = 0
    
    for i in range(1, 6):
        try:
            #find message text
            message_text = message_elements[-i].find_element(By.TAG_NAME, "p").text
            if "Waiting for message to decrypt" not in message_text:
                messages.append(message_text)
        except:
            errors += 1
    
    if errors >= 2:
        print("could load messages")
    if len(messages) == 0:
        print("Decrypting problem!")

    return messages

messages = get_last_n_messages(message_elements)

Decrypting problem!


In [169]:
# generate next message based on previous messages
def generate_next_message(messages):
    next_message = "Hello"
    
    return next_message

next_message = generate_next_message(messages)

In [185]:
new_message_element = new_message_element.find_element(By.XPATH, "..").find_element(By.XPATH, "//div[@role='textbox']")

In [187]:
def send_keys(element, text):
    for letter in text:
        element.send_keys(letter)
        time.sleep(random.randint(1, 7)/100)

send_keys(new_message_element, next_message)

In [195]:
send_message_element = driver.find_element(By.XPATH, "//button[@type='button' and @role='button' and @data-testid='submit']")
send_message_element.click()

In [ ]:
# write a message END

In [82]:
message_text_xpath = get_full_xpath_element(driver, element)

In [84]:
message_text_xpath

'/html/body/div/span/span/div[2]/div[2]/div[3]/div/div[2]/div/div/div/span/span/span[2]/span/span/span/div/div/p/span/span/span'

In [92]:
message_box_xpath = message_text_xpath[:message_text_xpath.find("span[")-1]
send_button_element = driver.find_element("xpath", message_box_xpath).find_elements("xpath", ".//button")[0]

In [94]:
send_button_element.click()

In [ ]:
message_box_xpath = message_text_xpath[:message_text_xpath.rfind("span[")-1]
send_button_element = driver.find_element("xpath", message_box_xpath).find_elements("xpath", ".//alt")[0]

In [212]:
# get daily points START

In [40]:
def get_daily_points(driver):
    # click on beaver icon
    towns_points_element = driver.find_element("xpath", '//img[@alt="Towns Points"]')
    towns_points_element.click()
    
    # find element inside beaver box
    beaver_text_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Towns Points')]")))
    
    # find beaver_box element
    beaver_text_xpath = get_full_xpath_element(driver, beaver_text_element)
    beaver_box_xpath = beaver_text_xpath[:beaver_text_xpath.rfind("div[") - 1]
    beaver_box_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, beaver_box_xpath)))
    
    # find and click on beaver_belly element
    beaver_belly_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, ".//canvas")))
    beaver_belly_element = beaver_box_element.find_element("xpath", ".//canvas").find_element("xpath", "..").find_element("xpath", ".//span")
    beaver_belly_element.click()
    
    # find and click on pay_with_ETH button
    pay_with_ETH_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Pay with ETH')]")))
    pay_with_ETH_element.click()
    
    # wait untill the tx will be processed
    text_to_wait = "You've earned"
    WebDriverWait(driver, 40).until(EC.visibility_of_element_located((By.XPATH, f'//*[contains(text(), "{text_to_wait}")]')))

get_daily_points(driver)
# TODO: check on if everything is already opened...

In [41]:
# get daily points END (FINISHED)

In [ ]:
# create new town START

In [83]:
from discord_names import unusual_discord_usernames

def generate_username():
    client = OpenAI(api_key=OPENAI_API)
    
    random_unusual_discord_usernames = random.sample(unusual_discord_usernames, 5)
    content_message = f"""
    Generate discord username.
    Provide only the generated name.
    
    Here are the examples:
    Generate discord username. Answer: {random_unusual_discord_usernames[0]}
    Generate discord username. Answer: {random_unusual_discord_usernames[1]}
    Generate discord username. Answer: {random_unusual_discord_usernames[2]}
    Generate discord username. Answer: {random_unusual_discord_usernames[3]}
    Generate discord username. Answer: {random_unusual_discord_usernames[4]}

    Now generate new one similar to the names in the examples.
    Generate discord username. Answer: """
    
    messages = [{"role": "user", "content": content_message}]
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating response from OpenAI: {e}")
        return None
        
username = generate_username()
username

'MysticPangolin'

In [84]:
from discord_names import unusual_discord_channels

def generate_town_name():
    client = OpenAI(api_key=OPENAI_API)
    
    random_unusual_discord_channels = random.sample(unusual_discord_channels, 5)
    content_message = f"""
    Generate discord channel name.
    Provide only the generated name.
    
    Here are the examples:
    Generate discord channel name. Answer: {random_unusual_discord_channels[0]}
    Generate discord channel name. Answer: {random_unusual_discord_channels[1]}
    Generate discord channel name. Answer: {random_unusual_discord_channels[2]}
    Generate discord channel name. Answer: {random_unusual_discord_channels[3]}
    Generate discord channel name. Answer: {random_unusual_discord_channels[4]}

    Now generate new one similar to the names in the examples.
    Generate discord channel name. Answer: """
    
    messages = [{"role": "user", "content": content_message}]
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error generating response from OpenAI: {e}")
        return None
        
town_name = generate_town_name()
town_name

'FrostedFalconField'

In [85]:
def generate_town_logo(town_name):
    client = OpenAI(api_key=OPENAI_API)

    basic_colors = ["red", "blue", "green", "yellow", "orange", "purple", "black", "white", "brown", "gray"]
    random_colors = random.sample(basic_colors, 2)
    
    response = client.images.generate(
        model="dall-e-2",
        prompt=f"A modern discord channel logo with the name {town_name} with minimalist design, {random_colors[0]} and {random_colors[1]} color theme",
        n=1,  # Number of images to generate
        size="256x256"  # Image size
    )
    
    # Get the generated image URL
    logo_url = response.data[0].url

    # Download and save the image
    path_to_logos = "towns_images"
    num_of_existed_logos = len(os.listdir(path_to_logos))
    
    logo_path = os.path.join(path_to_logos, f"town_logo_{num_of_existed_logos}.png")
    logo_data = requests.get(logo_url).content
    with open(logo_path, "wb") as file:
        file.write(logo_data)

    abs_logo_path = os.path.join(os.getcwd(), logo_path)

    return abs_logo_path
    
generate_town_logo(town_name)

'/home/asevlad/program_files/github_asevlad/towns_selenium/towns_images/town_logo_15.png'

In [ ]:
def save_town_link(town_link, town_type):
    towns_folder = "towns_links"
    if town_type == "free":
        towns_links_path = os.path.join(towns_folder, "free_towns.txt")
    elif town_type == "dynamic":
        towns_links_path = os.path.join(towns_folder, "dynamic_towns.txt")
    elif town_type == "state":
        towns_links_path = os.path.join(towns_folder, "state_towns.txt")

    with open(towns_links_path, 'a') as file:
        file.write("\n" + town_link)

In [86]:
def create_new_free_town():    
    driver.get("https://app.towns.com/t/new")
    
    # generate name and imamge for town
    town_name = generate_town_name()
    logo_path = generate_town_logo(town_name)

    # provide town_logo
    input_image_element = driver.find_element(By.XPATH, '//input[@type="file"]')
    input_image_element.send_keys(logo_path)
    driver.implicitly_wait(0.5)
    
    # provide town_name
    imput_town_name_element = driver.find_element(By.XPATH, "//input[@placeholder='Town Name']")
    imput_town_name_element.send_keys(town_name)
    time.sleep(3)
    
    # click next_button
    next_button_element = driver.find_element(By.XPATH, "//button[@data-testid='next-button']")
    next_button_element.click()
    time.sleep(3)
    
    # click Free button
    free_town_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Free')]")))
    free_town_element.click()
    time.sleep(3)

    # click create button
    create_town_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='create-town-button']")))
    create_town_element.click()
    
    # wait till creation
    while True:
        element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Give it a moment. Building a town takes a village')] | //*[contains(text(), 'You created')]")))
        
        if "Give it a moment. Building a town takes a village" in element.text:
            pass
        else:
            break

        time.sleep(2)

    # enter username if needed
    username_elements = driver.find_elements(By.XPATH, "//input[@data-testid='town-username-input']")
    if len(username_elements) != 0:
        username_element = username_elements[0]
        if username_element.get_attribute("value") == "":
            username = generate_username()
            username_element.send_keys(username)

    # save town_link
    town_link = driver.current_url
    town_link = town_link[:town_link.rfind("/channels")]

    save_town_link(town_link, "free")

    return town_link

In [ ]:
def create_new_dynamic_town():
    driver.get("https://app.towns.com/t/new")
    
    # generate name and imamge for town
    town_name = generate_town_name()
    logo_path = generate_town_logo(town_name)

    # provide town_logo
    input_image_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, '//input[@type="file"]')))
    input_image_element.send_keys(logo_path)
    driver.implicitly_wait(0.5)
    
    # provide town_name
    imput_town_name_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//input[@placeholder='Town Name']")))
    imput_town_name_element.send_keys(town_name)
    time.sleep(3)
    
    # click next_button
    next_button_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='next-button']")))
    next_button_element.click()
    time.sleep(3)
    
    # click Free button
    free_town_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Free')]")))
    free_town_element.click()
    time.sleep(3)

    # click create button
    create_town_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='create-town-button']")))
    create_town_element.click()
    
    # wait till creation
    while True:
        element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Give it a moment. Building a town takes a village')] | //*[contains(text(), 'You created')]")))
        
        if "Give it a moment. Building a town takes a village" in element.text:
            pass
        else:
            break

        time.sleep(2)

    # enter username if needed
    username_elements = driver.find_elements(By.XPATH, "//input[@data-testid='town-username-input']")
    if len(username_elements) != 0:
        username_element = username_elements[0]
        if username_element.get_attribute("value") == "":
            username = generate_username()
            username_element.send_keys(username)

    # save town_link
    town_link = driver.current_url
    town_link = town_link[:town_link.rfind("/channels")]

    save_town_link(town_link, "free")

    return town_link

In [ ]:
def create_new_dynamic_town(town_type, cost=0):
    driver.get("https://app.towns.com/t/new")
    
    # generate name and imamge for town
    town_name = generate_town_name()
    logo_path = generate_town_logo(town_name)

    # provide town_logo
    input_image_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, '//input[@type="file"]')))
    input_image_element.send_keys(logo_path)
    driver.implicitly_wait(0.5)
    
    # provide town_name
    imput_town_name_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//input[@placeholder='Town Name']")))
    imput_town_name_element.send_keys(town_name)
    time.sleep(3)
    
    # click next_button
    next_button_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='next-button']")))
    next_button_element.click()
    time.sleep(3)

    if town_type == "free":
        pass
        
    elif town_type == "dynamic":
        # click Paid option
        paid_town_element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Paid')]")))
        paid_town_element.click()
    
        # click Dynamic option
        dynamic_town_element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Dynamic')]")))
        dynamic_town_element.click()
        time.sleep(3)
        
    elif town_type == "state":
        
        
    # click Create button
    create_town_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='create-town-button']")))
    create_town_element.click()
    
    # wait till creation
    while True:
        element = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Give it a moment. Building a town takes a village')] | //*[contains(text(), 'You created')]")))
        
        if "Give it a moment. Building a town takes a village" in element.text:
            pass
        else:
            break

        time.sleep(2)

    # enter username if needed
    username_elements = driver.find_elements(By.XPATH, "//input[@data-testid='town-username-input']")
    if len(username_elements) != 0:
        username_element = username_elements[0]
        if username_element.get_attribute("value") == "":
            username = generate_username()
            username_element.send_keys(username)

    # save town_link
    town_link = driver.current_url
    town_link = town_link[:town_link.rfind("/channels")]

    save_town_link(town_link, "free")

    return town_link

In [152]:
driver.get("https://app.towns.com/t/new")

# generate name and imamge for town
town_name = generate_town_name()
logo_path = generate_town_logo(town_name)

# provide town_logo
input_image_element = driver.find_element(By.XPATH, '//input[@type="file"]')
input_image_element.send_keys(logo_path)
driver.implicitly_wait(0.5)

# provide town_name
imput_town_name_element = driver.find_element(By.XPATH, "//input[@placeholder='Town Name']")
imput_town_name_element.send_keys(town_name)
time.sleep(3)

# click next_button
next_button_element = driver.find_element(By.XPATH, "//button[@data-testid='next-button']")
next_button_element.click()
time.sleep(3)

In [156]:
# paid_town_element = WebDriverWait(driver, 20).until(
#         EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Paid')]")))
# paid_town_element.click()

fixed_town_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Fixed')]")))
fixed_town_element.click()

In [160]:
# click create button
cost_town_element = WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.XPATH, "//input[@name='slideMembership.membershipCost']")))
cost_town_element.send_keys(str(cost))

In [162]:
# click create button
create_town_element = WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.XPATH, "//button[@data-testid='create-town-button']")))
create_town_element.click()

In [232]:
# paid_town_element = WebDriverWait(driver, 90).until(
#         EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Paid')]")))
# paid_town_element.click()

In [228]:
free_town_element = WebDriverWait(driver, 90).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Free')]")))
free_town_element.click()

In [230]:
create_town_element = driver.find_element(By.XPATH, "//button[@data-testid='create-town-button']")
create_town_element.click()

In [236]:
# wait till creation
while True:
    element = WebDriverWait(driver, 90).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Give it a moment. Building a town takes a village')] | //*[contains(text(), 'You created')]")))
    if "Give it a moment. Building a town takes a village" in element.text:
        pass
    else:
        break

In [244]:
town_link = driver.current_url
town_link = town_link[:town_link.rfind("/channels")]
town_link

'https://app.towns.com/t/0x0f954f52b927314c120c6c111ceaafc96a6f6ba1'

In [ ]:
# create new town END

In [ ]:
# Update Towns START

In [83]:
def update_towns(driver):
    update_towns_p_element = None

    # try to find elemement with text 'Update Towns'
    try:
        update_towns_p_element = driver.find_element(By.XPATH, "//*[contains(text(), 'Update Towns')]")
    except:
        print("No element 'Update Towns'")

    # if element exists
    if update_towns_p_element:
        # get xpath of element with text
        update_towns_p_xapth = get_full_xpath_element(driver, update_towns_p_element)
        
        # get xpath of button with this element
        update_towns_button_xpath = update_towns_element[:update_towns_element.rfind("button") + 6]

        # get button element
        update_towns_button_element = driver.find_element(By.XPATH, update_towns_button_xpath)

        # click
        update_towns_button_element.click()

update_towns(driver)

No element 'Update Towns'


'/html/body/div/span/span/div[2]/div[2]/div[2]/div/div/span[2]/div/span/button/span/p'

In [351]:
def write_review(driver):
    # open review tab
    review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Review This Town')]")))
    review_element.click()

    # click write_review button to open box with writing review
    write_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Write a Review')]")))
    write_review_element.click()

    # find an element in the box
    your_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, ".//*[contains(text(), 'Your Review')]")))
    your_review_element_xpath = get_full_xpath_element(driver, your_review_element)
    
    # find xpath of the review box
    review_box_xpath = your_review_element_xpath[:your_review_element_xpath.rfind("span[") - 1]
    review_box_element = driver.find_element("xpath", review_box_xpath)

    # find text_area
    review_element = review_box_element.find_element("xpath", ".//textarea")
    # generate review
    review_text = generate_review()
    # send text to the textarea
    review_element.send_keys(review_text)

    time.sleep(1)
    
    # find rating element
    select_rating_element = driver.find_element("xpath","//*[contains(text(), 'Select your rating')]")
    rating = random.randint(4, 5)
    rating_element = select_rating_element.find_element("xpath", "..").find_elements("xpath", ".//div")[rating - 1]
    # rate town
    rating_element.click()

    time.sleep(1) # sleep between ac
    
    # click Post_review
    your_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Post Review')]")))
    post_review_element.click()
    
    # click on Pay_with_ETH
    your_review_element = WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Pay with ETH')]")))
    pay_with_ETH_element.click()
    
    # Wait till review will be submitted
    driver.find_element("xpath", "//*[contains(text(), 'Review posted successfully')]")

    time.sleep(1)
    
def send_keys(element, text):
    for letter in text:
        review_element.send_keys(letter)
        time.sleep(random.randint(1, 7)/100)
    
def generate_review():
    return "great town. I guess it will grow and grow"

In [359]:
# find an element in the box
your_review_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, ".//*[contains(text(), 'Your Review')]")))
your_review_element_xpath = get_full_xpath_element(driver, your_review_element)

# find xpath of the review box
review_box_xpath = your_review_element_xpath[:your_review_element_xpath.rfind("span[") - 1]
review_box_element = driver.find_element("xpath", review_box_xpath)

# find text_area
review_element = review_box_element.find_element("xpath", ".//textarea")
# generate review
review_text = generate_review()
# send text to the textarea
review_element.send_keys(".")

In [285]:
review_box_element = driver.find_element("xpath", review_box_xpath)
review_box_element

<selenium.webdriver.remote.webelement.WebElement (session="2073a7c028f7f6350c00602064e23a3f", element="f.82CD5B98A8D811ED9B7922CAB8F04263.d.18191B6AE14928AE93115074CCCE6043.e.2117")>

<selenium.webdriver.remote.webelement.WebElement (session="2073a7c028f7f6350c00602064e23a3f", element="f.82CD5B98A8D811ED9B7922CAB8F04263.d.18191B6AE14928AE93115074CCCE6043.e.2756")>

In [343]:
def give_rating(driver):
    select_rating_element = driver.find_element("xpath","//*[contains(text(), 'Select your rating')]")
    rating = random.randint(4, 5)
    rating_element = select_rating_element.find_element("xpath", "..").find_elements("xpath", ".//div")[rating - 1]
    rating_element.click()

In [345]:
rating

5

In [3]:
# AdsPower start profile START

In [5]:
ADS_API = os.getenv("ADS_API")

def open_ads_pwer_profile(profile_id):
    # Adspower API details
    API_URL = "http://127.0.0.1:50325"  # Change if Adspower runs on a different port

    # Start the Adspower profile
    start_url = f"{ADS_API}/api/v1/browser/start?user_id={profile_id}"
    response = requests.get(start_url).json()

    ws_url = response["data"]["ws"]["selenium"]
    print(f"WebSocket Debugging URL: {ws_url}")

    # Connect Selenium to the running Adspower profile
    options = webdriver.ChromeOptions()
    options.debugger_address = ws_url.replace("ws://", "").replace("/devtools/browser/", "")

    service = Service("/home/asevlad/program_files/github_asevlad/towns_selenium/chromedriver")  # Ensure you have the correct path
    driver = webdriver.Chrome(service=service, options=options)

    # Test by opening a website
    driver.get("https://www.google.com")

    return driver

driver = open_ads_pwer_profile("kw2ov4m")

NameError: name 'os' is not defined

In [ ]:
# AdsPower start profile END (FINISHED)